# 

In [1]:
import pandas as pd
from bioinfokit.analys import Fasta
import os

In [2]:
ont = ["K10B", "K10f", "LK_1.3.fasta"]
genome = ""
te = "Alex1_Alex3.fasta"

In [3]:
for file in ont:
    os.system(f"seqtk seq -a ont/{file}.fastq.gz > ont/{file}.fasta")
#!seqtk seq -a ont/K10B.fastq.gz > ont/K10B.fasta
for file in ont:
    os.system(f"makeblastdb -in ont/{file}.fasta -dbtype nucl -out ont/{file}")
#!makeblastdb -in ont/K10B.fasta -dbtype nucl -out ont/K10B
!mkdir -p blast
for file in ont:
    os.system(f"blastn -num_threads 20 -outfmt 6 -query te/{te} -db ont/{file} -out blast/TE_{file}.bl -dust no -perc_identity 0.9")
#!blastn -num_threads 20 -outfmt 6 -query te/Alex1_Alex3.fasta -db ont/K10B -out blast/Alex_K10B.bl -dust no -perc_identity 0.9

[E::stk_seq] failed to open the input file/stream.




Building a new DB, current time: 03/19/2024 20:38:19
New DB name:   /home/kira/test2/ont/K10B
New DB title:  ont/K10B.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kira/test2/ont/K10B
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 6939343 sequences in 211.099 seconds.




Building a new DB, current time: 03/19/2024 20:41:52
New DB name:   /home/kira/test2/ont/K10f
New DB title:  ont/K10f.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 4775182 sequences in 141.251 seconds.




Building a new DB, current time: 03/19/2024 20:44:14
New DB name:   /home/kira/test2/ont/LK_1.3.fasta
New DB title:  ont/LK_1.3.fasta.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B


BLAST options error: File ont/LK_1.3.fasta.fasta is empty
FASTA-Reader: Ignoring invalid residues at position(s): On line 109: 14-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 110: 1-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 111: 1-47
FASTA-Reader: Ignoring invalid residues at position(s): On line 114: 45-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 115: 1-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 116: 1-17
FASTA-Reader: Ignoring invalid residues at position(s): On line 142: 15-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 143: 1-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 144: 1-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 145: 1-42
FASTA-Reader: Ignoring invalid residues at position(s): On line 109: 14-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 110: 1-60
FASTA-Reader: Ignoring invalid residues at positio

In [10]:
headers = ['qseqid', 'sseqid',  'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'expect value', 'bitscore']

In [11]:
headers_dict = {i[0]:i[1] for i in enumerate(headers)}

In [12]:
def read_fasta(filename):
    return pd.read_csv(filename, sep="\t", header=None).rename(headers_dict, axis=1)
def save_fasta(df, filename):
    df.to_csv(filename, sep='\t', index=False, encoding='utf-8', header=False)

In [113]:
data = read_fasta("blast/TE_K10B.bl")

In [114]:
alex1 = data[data.qseqid.str.contains('Alex1')]
plus_alex1 = alex1[alex1.sstart < alex1.send]
minus_alex1 = alex1[alex1.sstart > alex1.send]

In [115]:
alex3 = data[data.qseqid.str.contains('Alex3')]
plus_alex3 = alex3[alex3.sstart < alex3.send]
minus_alex3 = alex3[alex3.sstart > alex3.send]

In [116]:
len(alex1) + len(alex3)

1732

In [117]:
plus_alex1.head(2)

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,expect value,bitscore
902,Alex1_chr1:18795335-18799058,2c40afb0-7c90-4ddf-847c-a587e291d4a9,94.235,2810,67,76,949,3724,2130,4878,0.0,4204.0
903,Alex1_chr1:18795335-18799058,2c40afb0-7c90-4ddf-847c-a587e291d4a9,95.327,535,7,15,421,949,612,1134,0.0,833.0


In [118]:
bed_df = plus_alex1[['sseqid', 'sstart', 'send', 'qseqid']].sort_values(by=['sseqid', 'sstart', 'send'])


In [119]:
bed_df.head()

,sseqid,sstart,send,qseqid
1083,0033bcbe-f78a-48ec-9149-88b241b8febb,3445,3714,Alex1_chr1:18795335-18799058
1082,0033bcbe-f78a-48ec-9149-88b241b8febb,3445,4367,Alex1_chr1:18795335-18799058
1081,0033bcbe-f78a-48ec-9149-88b241b8febb,5341,8025,Alex1_chr1:18795335-18799058
1084,0033bcbe-f78a-48ec-9149-88b241b8febb,7757,8025,Alex1_chr1:18795335-18799058
1492,012fd23f-1632-4f4a-8428-eb3c56914692,36,256,Alex1_chr1:18795335-18799058


In [120]:
save_fasta(bed_df, 'raw_first_blast.bed')

In [121]:
!bedtools merge -i raw_first_blast.bed -d 1500 -c 4 -o collapse  > first_blast.bed.merged

In [122]:
merged = pd.read_csv(
    "first_blast.bed.merged", sep="\t", header=None).rename(
    {0:'sseqid', 1:'sstart', 2:'send', 3:'qseqid'}, axis=1
)

In [123]:
len(merged)

295

In [124]:
merged.head(2)

,sseqid,sstart,send,qseqid
0,0033bcbe-f78a-48ec-9149-88b241b8febb,3445,8025,"Alex1_chr1:18795335-18799058,Alex1_chr1:187953..."
1,012fd23f-1632-4f4a-8428-eb3c56914692,36,433,"Alex1_chr1:18795335-18799058,Alex1_chr1:187953..."


In [212]:
filtered_first_bed = merged[(merged.send - merged.sstart) > 4000]

In [213]:
pd.options.mode.copy_on_write = True
filtered_first_bed['end'] = filtered_first_bed['send'] + 4000
filtered_first_bed['start'] = filtered_first_bed['sstart'] - 4000
save_fasta(filtered_first_bed[['sseqid', 'sstart', 'send']], 'normal.txt')


In [222]:
!seqtk subseq Alex_K10B.masked.fasta cutted2.txt > cutted_ont.fasta

In [215]:
!bedtools maskfasta -fi filtered_ont.fasta -bed normal.txt -fo Alex_K10B.masked.fasta
!seqkit fx2tab --length --name  Alex_K10B.masked.fasta > max_lenght.txt
max_len = pd.read_csv(
    'max_lenght.txt', sep="\t", header=None).rename(
    {0:'sseqid', 1:'max_len'}, axis=1).sort_values(by=['sseqid'])

In [218]:
#sseqid bo query to po prostu te w tym przypadku
save_fasta(filtered_first_bed[['sseqid', 'start', 'end']], 'cutted.txt')

mask_min = filtered_first_bed.start < 0
filtered_first_bed.loc[mask_min, 'start'] = 0
filtered_first_bed = pd.merge(filtered_first_bed, max_len, how='left', on="sseqid")
mask_max = filtered_first_bed.end > filtered_first_bed.max_len
filtered_first_bed.loc[mask_max, 'end'] = filtered_first_bed.max_len
save_fasta(filtered_first_bed[['sseqid', 'start', 'end']], 'cutted2.txt')


In [135]:
!mv output.fasta new2.fasta

# Second part of analysis

In [159]:
# Mask genome fasta
# Blast Alex to genome
#!blastn -num_threads 20 -outfmt 6 -query te/Alex1_Alex3.fasta -db genome/DCARv3.4 -out TE_genome.bl -perc_identity 0.8 -max_target_seqs 4
tex = read_fasta("TE_genome.bl")
# Mask genome fasta and save

In [165]:
#save_fasta(tex[["sseqid", "sstart", "send"]], "to_mask_genome.txt")
tex["start"] = tex['sstart'].where(tex['sstart'] <= tex["send"], other=tex['send'])
tex["stop"] = tex['sstart'].where(tex['sstart'] > tex["send"], other=tex['send'])
save_fasta(tex[["sseqid", "start", "end"]], "to_mask_genome.txt")
!bedtools maskfasta -fi genome/DCARv3.4.fa -bed to_mask_genome.txt -fo TE_genome.masked.fasta

Error: malformed BED entry at line 1. Start was greater than end. Exiting.


In [181]:
#okno = [record['sstart'], record['send'] if record['sstart'] >= record['send'] else None for record in tex]
#df['Conditioned_Numbers'] = df['Numbers'].where(df['Numbers'] > threshold, other='Below Threshold')
tex["start"] = tex['sstart'].where(tex['sstart'] <= tex["send"], other=tex['send'])
tex["stop"] = tex['sstart'].where(tex['sstart'] > tex["send"], other=tex['send'])
save_fasta(tex[["sseqid", "sstart", "send"]], "to_mask_genome.txt")

0     33630943
1     33628329
2     33630944
3     33626408
4      3170546
5      3167932
6      3166011
7      3170546
8      3643763
9      3640954
10     9941242
11     9941242
12     7058957
13     7058957
14     8891436
15     8893386
16      274398
17     8888935
18     8893386
19    51008118
20    51004338
21    33449114
22    51008118
23    51003667
24    33448294
25    41118869
26    50999547
27    55484402
28    42723505
29    42725457
30    55480622
31    39559061
32    55484403
33    42721004
34    55479950
35    39557575
36    42725457
37    39557074
38    46100369
39    40035438
40    46102319
41    46097868
42    46102319
Name: sstart, dtype: int64

In [220]:
!makeblastdb -in genome/DCARv3.4.fa  -dbtype nucl -out genome/DCARv3.4



Building a new DB, current time: 03/19/2024 12:33:22
New DB name:   /home/kira/test2/genome/DCARv3.4
New DB title:  genome/DCARv3.4.fa
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 9 sequences in 5.68707 seconds.




In [225]:
!blastn -num_threads 20 -outfmt 6 -query Alex_K10B.masked.fasta -db genome/DCARv3.4 -out Alex_K10B_genome_without_cut.bl -perc_identity 0.8 -max_target_seqs 4

In [127]:
result = pd.read_csv("Alex_K10B_genome_without_cut2.bl", sep="\t", header=None).rename(headers_dict, axis=1)

In [128]:
result.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,expect value,bitscore
0,328ef23f-1ebb-43b9-a01d-329e58159e42:1655-13665,DCARv3_Chr3,91.896,4109,144,142,3,4000,55475646,55479676,0.000000e+00,5568.0
1,328ef23f-1ebb-43b9-a01d-329e58159e42:1655-13665,DCARv3_Chr3,90.789,4028,178,153,8098,12011,55483962,55487910,0.000000e+00,5204.0
2,328ef23f-1ebb-43b9-a01d-329e58159e42:1655-13665,DCARv3_Chr3,81.985,272,38,10,745,1010,63314975,63314709,1.710000e-54,220.0
3,328ef23f-1ebb-43b9-a01d-329e58159e42:1655-13665,DCARv3_Chr3,80.435,230,33,8,727,950,44998433,44998210,8.140000e-38,165.0
4,328ef23f-1ebb-43b9-a01d-329e58159e42:1655-13665,DCARv3_Chr5,89.423,208,18,3,745,950,9000422,9000627,3.630000e-66,259.0


In [129]:
len(result.send)

2489

In [130]:
def create_raport(df):
    df = df.sort_values(by=['sseqid', 'sstart', 'send'])
    start_te = df.send[:-1]
    start_te = start_te.reset_index(drop=True)
    end_te = df.sstart[1:]
    end_te = end_te.reset_index(drop=True)
    return pd.DataFrame({'qseqid':df.qseqid.values[0], 'sseqid':df.sseqid.values[0], 'start_te': start_te, 'end_te': end_te})

series_list = []
for qseqid in result.qseqid.unique():
    okno = result[result.qseqid == qseqid]
    for sseqid in okno.sseqid.unique():
        okno2 = okno[(okno.sseqid == sseqid)]
        df = create_raport(okno2)
        series_list.append(df)


In [131]:
result = pd.concat(series_list)
result.to_csv("report2", sep='\t', index=False, encoding='utf-8', header=False)

In [137]:
len(result.end_te - result.start_te)

2393

In [141]:
okno = result
okno['length'] = okno.end_te - okno.start_te

In [142]:
okno.to_csv("okno2", sep='\t', index=False, encoding='utf-8', header=True) # i add header

In [145]:
finish = okno[(okno.length < 7000) & (okno.length > 3500)]

In [149]:
save_fasta(finish, 'finish')

In [151]:
ni = finish.sort_values(by=['sseqid', 'start_te'])

In [157]:
finish.head(30)

,qseqid,sseqid,start_te,end_te,length
1,328ef23f-1ebb-43b9-a01d-329e58159e42:1655-13665,DCARv3_Chr3,55479676,55483962,4286
5,704088c5-6d42-48cd-9194-46b372ca3c70:1431-13987,DCARv3_Chr2,27040072,27046332,6260
16,704088c5-6d42-48cd-9194-46b372ca3c70:1431-13987,DCARv3_Chr2,32077888,32081616,3728
25,704088c5-6d42-48cd-9194-46b372ca3c70:1431-13987,DCARv3_Chr2,32211960,32216454,4494
27,704088c5-6d42-48cd-9194-46b372ca3c70:1431-13987,DCARv3_Chr2,32378020,32381751,3731
7,98c0c670-f831-449a-a2d7-6b50aad7d341:308-12808,DCARv3_Chr1,45816544,45822054,5510
8,98c0c670-f831-449a-a2d7-6b50aad7d341:308-12808,DCARv3_Chr1,45818829,45822962,4133
4,cc3fff78-da65-4c38-8572-20a8d9cd7a6c:5431-17936,DCARv3_Chr5,18498187,18502233,4046
5,cc3fff78-da65-4c38-8572-20a8d9cd7a6c:5431-17936,DCARv3_Chr2,20220379,20226348,5969
25,c89a6c1c-495b-4c98-bcb1-376e6b1b3fb4:1700-11976,DCARv3_Chr3,55479676,55484400,4724
